<a href="https://colab.research.google.com/github/MonitSharma/Xanadu-Codebook-Solutions/blob/main/Quantum_Codebook_E3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## E.3 Are you Shor this works?

![](https://codebook.xanadu.ai/images/shor.png)

In [2]:
import numpy as np

#### Codercise E.3.1.

It turns out that Shor's code can even decode multi-qubit Pauli errors. So let's create a function that can generate any error we please. We'll use this extensively going forward.

The function that is available to you is called error. It takes two keyword arguments

In [5]:
# Create a Z error on qubit 4
z_error_4 = error(error_type="Z", wires=[4])

# Create a YX error on qubits 8 and 0
yx_error_80 = error(error_type="YX", wires=[8, 0])

# Create a XXZZYY error on qubits 1, 2, 3, 4, 5, and 0.
xxzzyy_error_123450 = error(error_type="XXZZYY", wires=[1, 2, 3, 4, 5, 0])


### Codercise E.3.2.

Complete the shor QNode below such that it performs exactly what the diagram above shows. The state  that gets encoded is an arbitrary quantum state; it doesn't matter what it is. We've left it as an argument — state — to the shor function, where you can initialize  to be state by using qml.QubitStateVector.

In [ ]:
dev = qml.device("default.qubit", wires=9)

@qml.qnode(dev)
def shor(state, error_type, wires):
    """A quantum circuit that implements Shor's 9-qubit code

    Args:
        state (tensor): a numpy array representing a 1-qubit state: alpha |0> + beta |1>
                        This is used to initialize the 0th wire with qml.QubitStateVector
        error_type (str): for example, XX, YY, XZ, YZ.
        wires (list(int)): the wires the error acts on.

    Returns:
        (tuple(tensor, tensor)): the separate probability distributions over the 0th wire (|psi>)
        and all 8 ancillary qubits in that order.
    """

    ##################
    # YOUR CODE HERE #
    ##################

    qml.CNOT(wires=[0,3])
    qml.CNOT(wires=[0,6])

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=3)
    qml.Hadamard(wires=6)

    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[3,4])
    qml.CNOT(wires=[6,7])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[3,5])
    qml.CNOT(wires=[6,8])

    # apply the error
    for err in error(error_type=error_type, wires=wires):
        err

    ##################
    # YOUR CODE HERE #
    ##################
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[3,4])
    qml.CNOT(wires=[6,7])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[3,5])
    qml.CNOT(wires=[6,8])

    qml.Toffoli(wires=[2, 1, 0])
    qml.Toffoli(wires=[5, 4, 3])
    qml.Toffoli(wires=[8, 7, 6])

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=3)
    qml.Hadamard(wires=6)

    qml.CNOT(wires=[0,3])
    qml.CNOT(wires=[0,6])

    qml.Toffoli(wires=[6,3,0])


    return qml.probs(wires=0), qml.probs(wires=range(1, 9))


### Codercise E.3.3.

Complete the decoded function below that will check to see if, given an error, Shor's code can correctly decode the input state. You have access to the shor function from before, as well as a function called random_state that can be used to create a random 1-qubit state (we will use it to initialize ) with positive and real coefficients.

In [ ]:
def decoded(state, error_type, wires):
    """Tells us whether the state was decoded by Shor's code

    Args:
        state (tensor): a numpy array representing a 1-qubit state: alpha |0> + beta |1>
        error_type (str): for example, XX, YY, XZ, YZ.
        wires (list(int)): the wires the error acts on.

    Returns:
        (bool): True if state is decoded successfully, False if not.
    """
    prob0 = shor(state, error_type, wires)[0]

    # Check if the probability of |0> state is above a threshold (e.g., 0.9)
    # If it is, then consider the state as decoded successfully
    threshold = 0.5
    if prob0[0] >= threshold:
        return True
    else:
        return False

state = random_state()

single_qubit_errors = [["X", [i]] for i in range(9)]
single_qubit_errors += [["Y", [i]] for i in range(9)]
single_qubit_errors += [["Z", [i]] for i in range(9)]

decoded_list = []

for err in single_qubit_errors:
    decoded_list.append(decoded(state, *err))

print(decoded_list)


###Codercise E.3.4.a.

Complete the find_the_one function such that it outputs the index  corresponding to where the error syndrome is equal to 1.

In [ ]:
def find_the_one(syndrome):
    """Finds the entry in the error syndrome that is equal to 1.

    Args:
        syndrome (tensor): the output of the shor function

    Returns:
        (int): the index of the syndrome vector that is equal to 1. For example,
        if syndrome = [0, 0, 1, 0], then this function would return 2.
    """
    index = np.nonzero(syndrome)[0]  # Find the indices where the error syndrome is equal to 1

    if len(index) > 0:
        return index[0]  # Return the first index where the error syndrome is equal to 1


### Codercise E.3.4.b.

Now that we can analyze the error syndrome from Shor's code, let's look for unique error syndromes. In doing this, not only are the errors decodable, but for some we might be even able to tell what the error was.

In [ ]:
single_qubit_errors = [["X", [i]] for i in range(9)]
single_qubit_errors += [["Y", [i]] for i in range(9)]
single_qubit_errors += [["Z", [i]] for i in range(9)]

all_syndromes = []
unique_syndromes = []
degenerate_syndromes = {}

state = random_state()

for err in single_qubit_errors:
    ind = find_the_one(shor(state, *err)[1])

    if ind in unique_syndromes:
        # find other error that gave the same syndrome
        other_ind = np.where(np.array(all_syndromes) == ind)[0].item()
        unique_syndromes.remove(ind) # remove it from the unique syndromes
        degenerate_syndromes[ind] = [single_qubit_errors[other_ind], err]

    elif ind in degenerate_syndromes.keys():
        degenerate_syndromes[ind].append(err)

    else:
        unique_syndromes.append(ind)

    all_syndromes.append(ind)

################
# PRESS SUBMIT #
################

print(degenerate_syndromes)


### Codercise E.3.5.
 Determine if Shor's code can decode the following errors:

In [ ]:
state = random_state()

xxzz_error_0458 = ["XXZZ", [0, 4, 5, 8]]
xxyyzz_error_371820 = ["XXYYZZ", [3, 7, 1, 8, 2, 0]]
xyzxyzxyz_error_012345678 = ["XYZXYZXYZ", [0, 1, 2, 3, 4, 5, 6, 7, 8]]

errors = [xxzz_error_0458, xxyyzz_error_371820, xyzxyzxyz_error_012345678]

decoded_list = []

for err in errors:
    decoded_list.append(decoded(state, *err))

print(decoded_list)
